# EEG Viewer
![status](https://img.shields.io/badge/status-in%20progress-orange)



<div style="text-align: center;">
    <img src="./assets/230524_eeg-viewer.png" alt="eeg viewer preview" width="450"/>
</div>

## Summary

This workflow is intended to demonstrate the visualization of a set of 1D EEG timeseries with HoloViz and Bokeh tools.

For details specific to this workflow, such as goals, specifications, and bottlenecks, please see this workflow's [readme](./readme_eeg-viewer.md).

For a summary of EEG research, data, and software, please see [neuro/wiki/EEG-notes](https://github.com/holoviz-topics/neuro/wiki/EEG-notes).

## Imports and config

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Requirements</p>
    <p>This workflow notebook requires the <a href="./environment.yml">environment</a> specified in this workflow directory.</p>
</div>


In [ ]:
import numpy as np
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from neurodatagen.eeg import generate_eeg_powerlaw
from scipy.stats import zscore
import panel as pn; pn.extension()

## Generate simulated data

The `generate_eeg_powerlaw` function synthesizes EEG data as high-pass filtered pink noise power law time series by default. The function returns a 2D numpy array of synthetic EEG data (in microvolts) shaped as (number of channels, total samples), a 1D time array (in seconds), and a list of channel names. Parameters such as the high-pass filter factor (in Hz) and an amplitude scaling factor allow customization of the generated data.

In [ ]:
n_channels = 25
n_seconds = 30
fs = 512

data, time, channels = generate_eeg_powerlaw(n_channels, n_seconds, fs)

print(f'shape: {data.shape} (n_channels, samples) ')
data

## Visualize synthetic EEG data with minimap

In [ ]:
# Set vertical spacing for EEG traces to avoid visual overlap
spacing = 1.2
offset = np.max(np.abs(data)) * spacing

# Create a hv.Curve element per chan.
# Note: alternative is to call hv.Path once on offset-adjusted data, but 
# then we couldn't independently apply formating to the channels (which 
# we aren't doing yet, but we likely will in the future)
channel_curves = []
for i, channel_data in enumerate(data):
    channel_curves.append(
        hv.Curve((time, channel_data + (i * offset)), "Time").opts(
            color="black", line_width=1, tools=["hover"], shared_axes=False))

# Create mapping from yaxis location to ytick for each channel
# so we can have categorical-style labeling on a continuous axis.
# Note: this would/should change when we implement independent 
# coordinates.
yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

# create a hook to set a predefined selection of RangeToolLink
max_ch_disp = 10
max_t_disp = 5
range_opts = []

def xrange_hook(plot, element):
    plot.handles['x_range'].end = max_t_disp

def yrange_hook(plot, element):
    plot.handles['y_range'].end = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
    
if time.max() > max_t_disp:
    range_opts.append(xrange_hook)

if len(channel_curves) > max_ch_disp:
    range_opts.append(yrange_hook)

# Create an overlay of curves
eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
    padding=0, xlabel="Time (s)", ylabel="Channel", #default_tools=['hover', 'pan', 'box_zoom', 'save', 'reset'],
    yticks=yticks, show_legend=False, hooks=range_opts, aspect=1.5, responsive=True,
    shared_axes=False)

# Get the y positions of the yticks to use as yaxis of minimap image
y_positions, _ = zip(*yticks)

# Compute z-scores across time for each channel
z_data = zscore(data, axis=1)

# Generate the zscored image for the minimap using the y tick positions from the eeg_viewer
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")

# Style the minimap 
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[], shared_axes=False)

# Create RangeToolLink between the minimap and the main EEG viewer 
# (quirk: apply to just one eeg trace and it will apply to all. see HoloViews #4472)
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"])

# Display vertically
# layout = (eeg_viewer + minimap).cols(1).opts(shared_axes=False, merge_tools=False)
# eeg_app = pn.Row(layout).servable() # too much spacing between plots in served app
eeg_app = pn.Column(pn.Row(eeg_viewer, min_height=500), minimap).servable() # BUG Panel #5315: rangetool is variably active in the bokeh toolbar on eeg viewer plot.. not respecting shared_axes=False
eeg_app


<div class="alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Tip:</p>
Hover near any border of the minimap, then click and drag to adjust the visible bounds in the EEG plot

</div>

## Load and plot real data